# MemoryBanditWorkflow を使った RAG の試験実装

(Version: 0.0.2.2)

先に MemoryBanditWorkflow というフレームワークを作った。それは迷路問題の一部として作り始めたが、汎用なものになるようこころがけた。それが実際汎用に使えることを示すために、RAG エージェントを作ってみようというのが今回の試みである。

アイデアの軌跡は↓にある。

\[cocolog:95619779\](2025年9月)  
《「LLM のメモリ機能を強制的に使うバンディットマシンの試験実装」と「LLM のメモリ機能とバンディット機能の試験実装」を行った。後者がメインの成果物で、メモリ機能の使用増加をどう強制するかから拡張したフレームワーク。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2025/09/post-8225e2.html

基本的にはメモリ機能とバンディット機能とワークフロー機能を持っているのが MemoryBanditWorkflow で、今回の物はそれをそのまま使って、RagAgent という子クラスを定義している。

なお、MemoryBanditWorkflow はセマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

MemoryBanditWorkflow の元となった迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、RAG エージェントといえばマルチエージェントを使うのが流行りのようだが、今回の物はリニアに順次、シングルスレッドで実行されていくだけである。マルチエージェントを敷衍するなら asyncio を使ったり、メモリにデータベースを使ったりすべきなのだが、そういうことはできる展望すらない。あくまで MemoryBanditWorkflow で RAG も実装できることを示そうとしたものなので、RAG としての新規性はない。


## 前回へのリンク

《experimental_rag_0.0.1.ipynb - GitHub Gist》  
https://gist.github.com/JRF-2018/6d7dda3eaf7743b91bca32037574bfc1

## 前回からの変更点

MemoryBanditWorkflow への変更は summarize_messages のエラーへの対策を少ししただけ。

0.0.1 からは RAG のループを少し変えて、一章ごと取材・執筆をするように指示を出し、バンディットが呼ばれやすくならないか試してみた。

0.0.2 から 0.0.2.2 への変更は、summarize_messages のエラー対策と、実行制御の全体像を AI に伝えるための show_execution_map ツールの導入。

## 結論

Gemini 2.5 Pro さんというコストのかかる遅いモデルを使った。やはり頭がいい者にやらせたかったから。前回(0.0.1)は修正しながら load して resume をやった。今回の 0.0.2 系列では、ほぼ一発出しでいけることもあったが、現在残っているログでは、途中、私の責任でなさそうなハングがあったりして、「二発出し」になっている。

一章ずつというわけにはいかなかったが、おおむね細かい単位で取材してくれて、バンディットが強制されやすくはなっていたと思う。前回に引き続き MemoryBanditWorkflow の実証実験としてはそこそこうまくいったと思う。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。コストもかかるしね \(^^;。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )


## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さんや Grok さん)に教わって作っています。

## 実装

まず、必要なライブラリを読み込みます。

In [1]:
!pip install -q -U langchain langgraph langchain-google-genai langmem duckduckgo-search langchain-community beautifulsoup4 ddgs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB

Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata

llm = ChatGoogleGenerativeAI(google_api_key=userdata.get('GOOGLE_API_KEY'), model="models/gemini-2.5-pro")
emb_llm = GoogleGenerativeAIEmbeddings(google_api_key=userdata.get('GOOGLE_API_KEY'), model='gemini-embedding-001')

ちゃんと Gemini にアクセスできるかテストします。

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="以下の質問に回答してください：{question}", input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"question": "Geminiモデルの特徴を教えてください"})
print(response)

/tmp/ipython-input-1709569937.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-1709569937.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": "Geminiモデルの特徴を教えてください"})


はい、承知いたしました。Googleが開発したAIモデル「Gemini」の特徴について、分かりやすくご説明します。

Geminiは、単なるテキスト生成AIではなく、**様々な種類の情報を同時に理解し、操作し、組み合わせることができる「マルチモーダルAI」**であることが最大の特徴です。

以下に、Geminiの主な特徴をまとめました。

### 1. ネイティブなマルチモーダル性
これがGeminiの最も革新的な点です。従来のAIは、テキストはテキスト、画像は画像と別々に学習してから連携させていましたが、Geminiは**最初からテキスト、画像、音声、動画、コードなどを一緒に学習**しています。

*   **何ができるか？**:
    *   **画像や動画を見て質問に答える**: ホワイトボードに書かれた物理の問題を写真に撮って「この問題を解いて」と頼んだり、料理動画を見せて「このレシピを教えて」と質問したりできます。
    *   **音声と画像を組み合わせる**: 楽器の絵を見せながら「この楽器の音を聴かせて」といった指示を理解できます。
    *   **多様な情報を統合して推論**: テキスト、グラフ、画像を組み合わせたレポートを読み込ませて、その内容を要約・分析させることができます。

### 2. 高度な推論能力
Geminiは、複雑な情報を理解し、多段階の思考を経て結論を導き出す高度な推論能力を持っています。

*   **何がすごいか？**:
    *   **専門的な知識**: 科学、数学、法律といった専門分野の難解な文章を理解し、質問に答えることができます。
    *   **隠れた意図の読解**: 単に書かれていることだけでなく、文脈や背景にある意図を読み取って、より的確な回答を生成します。
    *   **問題解決**: 複雑な数学の問題を、人間のように段階的に解き進めることができます。

### 3. 卓越したパフォーマンス
Geminiは、AIの性能を測る様々なベンチマークテストにおいて、GPT-4など他の主要なAIモデルを上回る、あるいは同等以上の非常に高いスコアを記録しています。特に、一般的な知識や問題解決能力を測る「MMLU」というテストでは、人間の専門家を初めて上回る性能を示しました。

### 4. 用途に応

埋め込みベクトルも試しておきます。

In [4]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.013715926557779312,
 -0.0006048799259588122,
 0.011018728837370872,
 -0.09579180181026459,
 0.006502930540591478]

基本的なモジュールを読み込みます。

In [5]:
import os
import math
import numpy as np
import random
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [6]:
RAG_AGENT_SAVE = "rag-agent.pickle"

ライブラリ集。

In [7]:
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy
from IPython.display import Markdown

# LangGraphのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import WebBaseLoader

INITIAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        ("user", "上記の会話の要約を作成してください:"),
    ]
)

EXISTING_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        (
            "user",
            "これまでの会話の要約です: {existing_summary}\n\n"
            "上記の新しいメッセージを考慮して、この要約を拡張してください:",
        ),
    ]
)

FINAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        # if exists
        ("placeholder", "{system_message}"),
        ("system", "これまでの会話の要約: {summary}"),
        ("placeholder", "{messages}"),
    ]
)


In [8]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance


def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str


MemoryBanditWorkflow は変わっていないです。長大な定義で申し訳ないです。

In [15]:
class MemoryBanditWorkflow:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file

        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = []

        self.access_unit = 1.0
        self.recent_reads = []

        self.workflows = {}
        self.workflow_current = "workflow:main"
        self.workflow_next = None
        self.privileged_tool_names = []

        self.init_memories()
        self.init_workflows()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?memory:(\d+)\]?', id_or_num)
            if m:
                return f"memory:{m.group(1)}"
            if id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _normalize_workflow_id_sub(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"workflow:{id_or_num}"
        if id_or_num in ["current", "main"]:
            return f"workflow:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?workflow:(\d+|main|current)\]?(?:.+)?', id_or_num.strip())
            if m:
                return f"workflow:{m.group(1)}"
            if id_or_num.isdigit():
                return f"workflow:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def normalize_workflow_id(self, id_or_num):
        r = self._normalize_workflow_id_sub(id_or_num)
        if r == "workflow:current":
            return self.workflow_current
        return r

    def _replace_tools (self, from_tools, to_tools):
        tool_names = [x.name for x in to_tools]
        return [x for x in from_tools
                if x.name not in tool_names] + to_tools

    def _create_tools (self):
        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"ツール(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent(top_n: int = 10) -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > top_n:
                    sorted_memories = sorted_memories[:top_n]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_list_random(top_n: int = 10) -> Dict[str, Any]:
            """
            メモリをランダムにリストします。
            """

            keys = list(self.memories.keys())
            if len(keys) > top_n:
                keys = random.sample(keys, top_n)
            if keys:
                values = [self.memories[k] for k in keys]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in values]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: メモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "", workflow_id: str = "workflow:current") -> str:
            """
            ツールの利用を強制するためのバンディットを予約します。

            Args:
                tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                aux_prompt: 実行の詳細を示すための補助プロンプト または ""
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            tool_names = re.split(r"\s+or\s+|\s+OR\s+", tool_name)
            prohibited = set(self.privileged_tool_names) & set(tool_names)
            if prohibited:
                return f"失敗。{repr(prohibited)} は登録できません。"
            all_tools = [x.name
                         for x in self._create_tools()['all_tools']]
            if not any (x in all_tools for x in tool_names):
                return f"失敗。{tool_name} は有効なツールでありません。"

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには tool_name exec_mode aux_prompt のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'{tool_name}' は書き換えられません。"
            else:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            memory_read ツールの利用を強制するためのバンディットを予約します。

            Args:
                memory_id: memory_read するための memory_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode memory_id のすべてを一致させて指定する必要があります。"
                x = {'pin': None, 'arg': memory_id}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'memory_read {memory_id}' は書き換えられません。"
            else:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_read): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_workflow(workflow_id_to_schedule: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            workflow_do ツールの利用を強制するためのバンディットを予約します。

            Args:
                workflow_id_to_schedule: workflow_do するための workflow_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            workflow_id2 = self.normalize_workflow_id(workflow_id_to_schedule)
            if workflow_id2 not in self.workflows:
                return f"失敗。{workflow_id2} は有効なワークフローではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "workflow_do" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == workflow_id2:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode workflow_id_to_schedule のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': workflow_id2
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'workflow_do {workflow_id2}' は書き換えられません。"
            else:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_workflow): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_list(workflow_id: str =  "workflow:current")  -> Dict[str, Any]:
            """
            登録されているバンディットのスタックを返します。

            Args:
                workflow_id: バンディットのスタックが登録されたワークフローを指定します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            return {'status': 'success',
                    'result': self.workflows[workflow_id]['stack']}

        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが完了したことを宣言します。サブワークが完了したときのみ使ってください。
            """

            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def workflow_do(workflow_id: str)  -> str:
            """
            ワークフローを実行します。
            """

            self.workflow_next = None
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'e' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は子として実行できません。"
            self.workflow_next = workflow_id
            title = self.workflows[workflow_id]['title']
            return f"成功。この後、{workflow_id} 「{title}」を実行していきます。"

        @tool
        def workflow_list()  -> Dict[str, Any]:
            """
            登録されているすべてのワークフローの id とその title および pin 状況を返します。
            """

            return {'status': 'success',
                    'result': list(self.workflows.values())}

        @tool
        def workflow_show_current()  -> str:
            """
            現在実行中のワークフローのタイトル、および、強制しているバンディットの情報を表示します。
            """

            wtitle = self.workflows[self.workflow_current]['title']
            mes = dedent(f"""\
            現在のワークフロー: {self.workflow_current} 「{wtitle}」
            現在のバンディットの指示: 「{self.cur_bandit_prompt}」
            現在のバンディット: {repr(self.cur_bandit)}
            現在のバンディットの done 数: {self.cur_bandit_done}
            """)
            print(f"ツール(workflow_show_current): {mes}")
            return mes


        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: str)  -> str:
            """
            新しいワークフローを定義し、その workflow_id を返します。

            Args:
                title: ワークフローの名前。
                bandits: 登録するバンディットのリスト。
                pin: 'w' は書換不能。'd' は削除不能。'wd' は書換＆削除不能。'' は書換＆削除可能。

            バンディットはそれぞれ辞書型、それを b とすると。
            b['tool_name'] は bandit_schedule での tool_name と同じ。
            b['exec_mode'] は bandit_schedule の exec_mode と同じ。
            b['aux_prompt'] は bandit_schedule の aux_prompt と同じ。
            b['prob'] は bandit_schedule の prob と同じ。
            b['times'] は bandit_schedule の times と同じ。
            b['arg'] は b['tool_name'] が 'memory_read' のとき memory:〜 という memory_id を指定できる。または、b['tool_name'] が 'workflow_do' のとき workflow:〜 という workflow_id を指定できる。
            b['pin'] は、None のとき削除または prob と times の書き換え可能。'stack' のときは prob と times の書き換え可能。'write' のときは書き換えも不能。
            """
            # AI に対する隠し属性として pin の 'e' は子として実行不能。

            for b in bandits:
                if not all(x in b for x in ['tool_name', 'exec_mode',
                                            'aux_prompt', 'times', 'prob']):
                    return f"失敗。有効な定義ではありません。"
                if 'arg' not in b:
                    b['arg'] = None
                if b['tool_name'] not in ["memory_read", "workflow_do"] \
                   and b['arg']:
                    return f"失敗。有効な定義ではありません。"
                if b['arg'] and b['tool_name'] == "memory_read":
                    b['tools_name'] = "read_tools"
                else:
                    b['tools_name'] = "default_tools"
                if 'pin' not in b:
                    b['pin'] = None
                if not (b['pin'] is None or b['pin'] == 'stack'
                        or b['pin'] == 'write'):
                    return f"失敗。不正な pin 値です。"
                tool_names = re.split(r"\s+or\s+|\s+OR\s+", b['tool_name'])
                prohibited = set(self.privileged_tool_names) & set(tool_names)
                if prohibited:
                    return f"失敗。{repr(prohibited)} は登録できません。"
                all_tools = [x.name
                             for x in self._create_tools()[b['tools_name']]]
                if not any (x in all_tools for x in tool_names):
                    return f"失敗。{b['tool_name']} は有効なツールでありません。"

            i = 1000
            while True:

                if f"workflow:{i}" not in self.workflows:
                    break
                i = i + 1
            new_id = f"workflow:{i}"

            self.workflows[new_id] = {'stack': bandits, 'pin': pin,
                                      'title': title}
            print(f"ツール(workflow_new): {repr(self.workflows[new_id])}")
            return f"成功。{new_id} を新規登録しました。"

        @tool
        def workflow_delete(workflow_id: str)  -> str:
            """
            ワークフローを削除します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} はありません。"
            if 'd' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は削除できません。"
            del self.workflows[workflow_id]
            return f"成功。{workflow_id} を削除しました。"

        r = {}
        r['null_tools'] = []
        r['read_tools'] = r['null_tools'] + [
            express_thought,
            update_plan, show_plan, update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent, memory_list_random,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            bandit_schedule, bandit_schedule_read, bandit_list,
            bandit_statistics,
            subwork_done,
            workflow_do, workflow_new, workflow_list,
            workflow_show_current, workflow_delete,
            bandit_schedule_workflow
        ]
        r['default_tools'] = r['read_tools'] + [
            memory_new, memory_update_string, memory_append_string,
            memory_delete
        ]
        r['all_tools'] = r['default_tools']
        return r

    def _create_agent (self, tools_name='default_tools'):
        tools = self._create_tools()[tools_name]
        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm, tools, prompt=self.system_prompt,
            checkpointer=InMemorySaver(),
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _summarize_messages(self):
        try:
            cur_top = self.messages[0]
            res = summarize_messages(
                self.messages,
                max_tokens=5000,
                max_summary_tokens=2000,
                running_summary=self.running_summary,
                model=self.llm,
                initial_summary_prompt=INITIAL_SUMMARY_PROMPT,
                existing_summary_prompt=EXISTING_SUMMARY_PROMPT,
                final_prompt=FINAL_SUMMARY_PROMPT,
            )
            self.messages = res.messages
            if self.messages[0].content != cur_top.content:
                print(f"新しい要約: {self.messages[0].content}")
            self.running_summary = res.running_summary
        except Exception as e:
            import traceback
            traceback.print_exc()
            self.running_summary = None
            self._sanitize_messages()

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対症療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def run (self, workflow_main_id):
        print("\n\n----------\n\n")
        self.messages2ids = []

        self.workflow_current = workflow_main_id
        bandits = copy.deepcopy(
            self.workflows[self.workflow_current]['stack']
        )
        working_bandit = None
        workflow_stack = []
        execed = []
        while True:
            while working_bandit is not None or bandits:
                if working_bandit is not None:
                    b, done, prev_done = working_bandit
                    working_bandit = None
                else:
                    b = bandits.pop()
                    done = 0
                    prev_done = True
                enforce = b['tool_name']
                aux_prompt = b['aux_prompt']
                tools_name = b['tools_name']
                memory_id = None
                workflow_id = None
                if b['arg'] and enforce == 'memory_read':
                    memory_id = b['arg']
                if b['arg'] and enforce == 'workflow_do':
                    workflow_id = b['arg']

                while done < b['times']:
                    if not random.random() < b['prob']:
                        done += 1
                        continue
                    if memory_id and memory_id not in self.memories:
                        done += 1
                        continue
                    if workflow_id and workflow_id not in self.workflows:
                        done += 1
                        continue
                    all_tools = [x.name for x in self._create_tools()[tools_name]]
                    tool_names = re.split(r"\s+or\s+|\s+OR\s+", enforce)
                    if not any (x in all_tools for x in tool_names):
                        done += 1
                        continue
                    if memory_id:
                        aux_prompt = f"{memory_id} を読んでください。"
                    if workflow_id:
                        aux_prompt = f"{workflow_id} を実行してください。"
                    self.cur_bandit = b
                    self.cur_bandit_done = done
                    self.cur_bandit_prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                    prompt = self.cur_bandit_prompt
                    if not prev_done:
                        prompt = "前回の指示がまだ完了してません。前回の指示: " + prompt
                    print(f"USER_INPUT: {prompt}")
                    self.messages.append(HumanMessage(prompt))
                    config = {"configurable": {"thread_id": "1"}}
                    app = self._create_agent(tools_name=tools_name)
                    self.access_unit = 0.3 if memory_id else 1.0
                    prev_done = False
                    try:
                        for chunk, metadata in app.stream(
                                {"messages": self.messages.copy()},
                                config=config,
                                stream_mode="messages",
                        ):
                            if chunk.id in self.messages2ids:
                                continue
                            self.messages.append(chunk)
                            if isinstance(chunk, ToolMessage):
                                print(f"ツール結果({chunk.name}): {short_repr(chunk.content)}")
                            if isinstance(chunk, ToolMessage) \
                               and chunk.name == "workflow_do":
                                if chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                   and (not workflow_id or workflow_id == self.workflow_next):
                                    done += 1
                                    prev_done = True
                                    execed.append(b)
                                    if not self.workflow_next:
                                        break
                                if not self.workflow_next:
                                    continue
                                workflow_stack.append((
                                    (b, done, prev_done),
                                    bandits,
                                    execed,
                                    self.workflow_current
                                ))
                                self.workflow_current = self.workflow_next
                                bandits = copy.deepcopy(self.workflows[self.workflow_current]['stack'])
                                working_bandit = None
                                execed = []
                                break
                            elif isinstance(chunk, ToolMessage) \
                               and chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                               and (not memory_id or memory_id == self.recent_reads[-1]['id']):
                                done += 1
                                prev_done = True
                                execed.append(b)
                                break
                        self._filterout_messages2()
                        self._summarize_messages()
                        print(f"エージェントの応答: {self.messages[-1].content}")
                    except GraphRecursionError as e:
                        print(f"Recursion Limit に到達しました。")
                        self._filterout_messages2()
                        self._summarize_messages()
                    except Exception as e:
                        print(f"エラーが発生しました: {e}")
                        import traceback
                        traceback.print_exc()
                        self._sanitize_messages()
                        #raise e

            for b in execed:
                for x in self.workflows[self.workflow_current]['stack']:
                    if x['tool_name'] == b['tool_name'] \
                       and x['exec_mode'] == b['exec_mode'] \
                       and x['aux_prompt'] == b['aux_prompt'] \
                       and x['arg'] == b['arg'] \
                       and x['exec_mode'] == "once":
                        if x['times'] > 0:
                            x['times'] -= 1
            self.workflows[self.workflow_current]['stack'] = [
                x for x in self.workflows[self.workflow_current]['stack']
                if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
            ]

            if not workflow_stack:
                break
            workflow_prev = self.workflow_current
            prev_title = self.workflows[workflow_prev]['title']
            working_bandit, bandits, execed, self.workflow_current \
                = workflow_stack.pop()
            cur_title = self.workflows[self.workflow_current]['title']
            mes = f"{workflow_prev} 「{prev_title}」から {self.workflow_current} 「{cur_title}」に復帰しました。"
            print(f"USER_INPUT: {mes}")
            self.messages.append(HumanMessage(mes))

    def listen_and_print (self, prompt):
        ans = None
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            self._summarize_messages()
            ans = response['messages'][-1].content
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        memories = [
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            },
            {
                'id': 'memory:9994',
                'title': 'keyword の増強',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                memory_list_random という機能を使って 5 つリストアップして、それぞれを読み、それぞれに適切なキーワードを付けることが可能なら 「keyword: 〜」文を append するように。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        workflow_main = [
            {
                'tool_name': 'memory_new',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "最近のやり取りを要約して書いてください。",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            # {
            #     'tool_name': 'subwork_done',
            #     'tools_name': 'default_tools',
            #     'exec_mode': 'persistent',
            #     'aux_prompt': "ワークフローのテストをします。適当なパラメータで workflow_new を実行し、それをすぐさま workflow_delete します。そして、bandit_schedule で、指示しているこのバンディットを削除します。完了したら subwork_done してください。",
            #     'arg': None,
            #     'prob': 1.0,
            #     'times': 1,
            #     'pin': None,
            # },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'workflow_do',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': "workflow:1000",
                'prob': 1.0/20,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows["workflow:main"] \
            = {'pin': 'de', 'stack': workflow_main, 'title': "メイン"}

        workflow_sub = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "memory:9994 を読みその指示を実行し、そのタスクが終ったら、subwork_done してください。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9994',
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]
        self.workflows["workflow:1000"] \
            = {'pin': 'wd', 'stack': workflow_sub, 'title': 'keyword 更新'}

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword.startswith("〜"):
                continue
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, result: Any)  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            result (Any): 処理結果のデータまたはエラーメッセージ。
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {short_repr(result)}")
            self.backend_status = status
            self.backend_result = result

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm2, tools, prompt=self.system_prompt2,
            checkpointer=InMemorySaver(),
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None
        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk, metadata in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="messages",
                ):
                    self.messages2.append(chunk)
                    self.messages2ids.append(chunk.id)
                    if isinstance(chunk, ToolMessage):
                        print(f"ツール結果2({chunk.name}): {short_repr(chunk.content)}")
                    if isinstance(chunk, ToolMessage) and chunk.name == "set_result":
                        break
                print(f"エージェントの応答: {self.messages2[-1].content}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                #raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


メインの RagAgent。

In [16]:
class RagAgent (MemoryBanditWorkflow):
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.thesis = {
            'title': "",
            'chapters': [{'title': '概要', 'text': ""}]
        }
        super().__init__(llm=llm, llm2=llm, emb_llm=emb_llm,
                         save_file=save_file)

        self.system_prompt = """\
あなたは賢い RAG エージェントです。論文(thesis)を書き上げていきます。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。

メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。

論文(thesis)は章(chapter)に分けて作っていきます。それぞれが Markdown 形式です。論文のタイトルは # 一つ、章のタイトルは ## で修飾されることになりますので、それに合わせた階層で書いてください。
"""

        self.system_prompt2 = """\
あなたは賢い RAG エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""
        self.subwork_done = True
        self.user_demand = ''
        self.current_work = ''
        self.current_state = 0

    def _create_tools (self):
        @tool
        def show_user_demand () -> str:
            """
            大目的を返します。
            """
            print(f"ツール(show_user_demand): {self.user_demand}")
            return self.user_demand

        @tool
        def show_current_work () -> str:
            """
            中目的すなわち現在の仕事を返します。
            """
            print(f"ツール(show_current_work): {self.current_work}")
            return self.current_work

        @tool
        def thesis_write_title (new_title: str) -> str:
            """
            論文のタイトルを更新します。
            """
            self.thesis['title'] = new_title
            mes = "論文のタイトルが更新されました。"
            print(f"ツール(thesis_write_title): {mes}: {new_title}")
            return mes

        @tool
        def thesis_show_title () -> str:
            """
            論文のタイトルを返します。
            """
            print(f"ツール(thesis_show_title): {self.thesis['title']}")
            return self.thesis['title']

        @tool
        def thesis_new_chapter (title: str, text: str) -> str:
            """
            新しい章を title と text で構成します。
            """

            x = {'title': title, 'text': text}
            num = len(self.thesis['chapters'])
            self.thesis['chapters'].append(x)
            print(f"ツール(thesis_new_chapter): {short_repr(x)}")
            return f"成功: 第{num}章を作りました。"

        @tool
        def thesis_write_chapter (
                chapter_num: int,
                new_title: str,
                new_text: str
        ) -> str:
            """
            章を new_title と new_text で置き換えます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            x = {'title': new_title, 'text': new_text}
            self.thesis['chapters'][chapter_num] = x
            print(f"ツール(thesis_write_chapter): {short_repr(x)}")
            return f"成功: 第{chapter_num}章を書き換えました。"

        @tool
        def thesis_delete_last_chapter () -> str:
            """
            最後の章を削除します。
            """
            num = len(self.thesis['chapters']) - 1
            if num <= 0:
                return f"失敗: これ以上削除できません。"
            self.thesis['chapters'].pop()
            return f"成功: 第{num}章を削除しました。"

        @tool
        def thesis_read_chapter(chapter_num: int) -> Union[Dict[str, str], str]:
            """
            指定された章を読み込みます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            return self.thesis['chapters'][chapter_num]

        @tool
        def thesis_list_chapters() -> List[str]:
            """
            章のタイトルのリストを返します。
            """
            return [x['title'] for x in self.thesis['chapters']]

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが(成功裡に)完了したことを宣言します。
            """
            self.subwork_done = True
            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def subwork_not_done()  -> str:
            """
            指示されたサブワークが完了してないことを宣言します。
            """
            self.subwork_done = False
            return f"成功。サブワークの未完了が宣言されました。"

        @tool
        def show_execution_map () -> str:
            """
            workflow より上位の実行制御の全体像とそこにおける現在の State を返します。
            """
            mes = dedent(f"""\
            大目的は「{self.user_demand}」です。

            次に実行制御の全体像を示します。State 0 からはじまります。指示があったとき subwork_done または subwork_not_done することで次の State に遷移します。

            State 0: 大目的に沿ってメモリ等に計画を立てます。完了時 State 1 へ。

            State 1: 大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していきます。完了時 State 2 へ。

            State 2: 大目的に沿ってメモリ等にある取材データが全章分、十分か確かめます。完了時 State 3 へ。未完了時 State 1 へ。

            State 3: 大目的に沿ってメモリ等にある取材データを元に論文(thesis)をまだ書いてない一章分、十分でない一章分を書きます。完了時 State 4 へ。

            State 4: 大目的に沿って取材データを元にした論文(thesis)が全章完成しているかをチェックします。最後にタイトルを設定し、第0章の概要を書き直します。完了時 State 6 へ。未完了時 State 5 へ。

            State 5: 大目的に沿ってまだ追加取材が必要なら再調査してメモリ等に記録します。完了時 State 3 へ

            State 6: 終了。完成！

            現在の State は {self.current_state} です。
            """)
            print(f"ツール(show_execution_map): {mes}")
            return mes


        @tool
        def get_web_page_content(url: str) -> str:
            """
            指定されたURLからWebページのコンテンツを取得して文字列として返します。
            調査でURLを見つけた後に使ってください。
            """
            try:
                loader = WebBaseLoader(url)
                docs = loader.load()
                content = " ".join([doc.page_content for doc in docs])
                print(f"ツール(get_web_page_content): 成功: {url}")
                # 長すぎる場合は適当に丸める
                if len(content) > 5000:
                    return content[:5000] + "... (コンテンツが長すぎるため省略)"
                return content
            except Exception as e:
                print(f"ツール(get_web_page_content): 失敗: {url}")
                return f"エラー: ページの読み込みに失敗しました。URL: {url}, エラー: {e}"

        search_tool = DuckDuckGoSearchResults(output_format="list", verbose=True)
        web_page_tool = get_web_page_content

        r = super()._create_tools()
        for k in r:
            if k != 'null_tools':
                r[k] = self._replace_tools(r[k], [
                    show_user_demand, show_current_work,
                    thesis_write_title, thesis_show_title,
                    thesis_new_chapter, thesis_write_chapter,
                    thesis_delete_last_chapter,
                    thesis_read_chapter, thesis_list_chapters,
                    subwork_done, subwork_not_done, show_execution_map,
                    search_tool, web_page_tool
                ])
        return r

    def main_loop (self, user_demand):
        self.user_demand = user_demand
        self.current_state = 0
        self.resume()

    def resume (self):
        if self.current_state == 0:
            self.execute_planning()
            self.current_state = 1
            self.save()
            print("\n\n----------\n\n")
        while self.current_state in [1, 2]:
            if self.current_state == 1:
                self.execute_investigation()
                self.current_state = 2
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 2:
                r = self.execute_check_of_investigation()
                if r:
                    self.current_state = 3
                else:
                    self.current_state = 1
                self.save()
                print("\n\n----------\n\n")
        while self.current_state in [3, 4, 5]:
            if self.current_state == 3:
                r = self.execute_writing()
                self.current_state = 4
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 4:
                r = self.execute_check_of_writing()
                if r:
                    self.current_state = 6
                else:
                    self.current_state = 5
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 5:
                self.execute_reinvestigation()
                self.current_state = 3
                self.save()
                print("\n\n----------\n\n")
        print("Done!")

    def write_loop (self):
        if self.current_state not in [3, 4, 5]:
            self.current_state = 3
        self.resume()

    def _execute (self, current_work, tool_name, aux_prompt):
        self.current_work = current_work
        user_input = dedent(f"""\
        大目的: 「{self.user_demand}」
        計画と方針: 「{self.plan}」
        スクラッチパッド: 「{self.scratchpad}」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「{self.current_work}」
        """)
        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        self.workflows['workflow:main']['stack'][0] = {
            'tool_name': tool_name,
            'tools_name': 'all_tools',
            'exec_mode': 'persistent',
            'aux_prompt': aux_prompt,
            'arg': None,
            'prob': 1.0,
            'times': 1,
            'pin': 'write'
        }
        self.run("workflow:main")

        return self.subwork_done

    def execute_planning (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等に計画を立ててください。",
            tool_name = 'subwork_done',
            aux_prompt = "計画を立て終ったら subwork_done してください。",
        )

    def execute_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していってください。",
            tool_name = 'subwork_done',
            aux_prompt = "一章分取材し終ったら subwork_done してください。",
        )

    def execute_check_of_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データが全章分、十分か確かめてください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "全章の取材が十分なら subwork_done してください。十分でないと判断したら subwork_not_done してください。",
        )

    def execute_writing (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データを元に論文(thesis)をまだ書いてない一章分、十分でない一章分を書いてください。ちなみに第0章は概要と決まっています。",
            tool_name = 'subwork_done',
            aux_prompt = "良い論文のうち一章分書けたら subwork_done してください。",
        )

    def execute_check_of_writing (self):
        return self._execute(
            current_work = "大目的に沿って取材データを元にした論文(thesis)が全章完成しているかをチェックしてください。最後にタイトルを設定し、第0章の概要を書き直してください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "良い論文が全章書け、タイトルを設定し、概要を書き直せたら subwork_done してください。まだ全章書けていなければ subwork_not_done してください。",
        )

    def execute_reinvestigation (self):
        return self._execute(
            current_work = "大目的に沿ってまだ追加取材が必要なら再調査してメモリ等に記録してください。",
            tool_name = 'subwork_done',
            aux_prompt = "取材し終ったら subwork_done してください。",
        )

    def init_memories (self):
        super().init_memories()
        memories = []
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        super().init_workflows()
        workflow_main = self.workflows['workflow:main']['stack']
        workflow_main = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'all_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "仮登録。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ] + workflow_main + [
            {
                'tool_name': 'show_execution_map',
                'tools_name': 'default_tools',
                'exec_mode': 'once',
                'aux_prompt': "",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': None
            }
        ]
        self.workflows['workflow:main']['stack'] = workflow_main


    def display_thesis (self):
        s = f"# {self.thesis['title']}\n\n"
        for i, x in enumerate(self.thesis['chapters']):
            if re.search(r"^\#\#", x['text']):
                pass
            elif i == 0 or re.search(r"[01-9]+章", x['title']):
                s += f"## {x['title']}\n\n"
            else:
                s += f"## 第{i}章 {x['title']}\n\n"
            s += f"{x['text']}\n\n"
        return Markdown(s)


クラスをインスタンシエート。

In [17]:
agent = RagAgent(llm=llm, llm2=llm, emb_llm=emb_llm, save_file=RAG_AGENT_SAVE)
agent.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [ ]:
agent = RagAgent.load(RAG_AGENT_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、動かしてみます。

In [18]:
agent.main_loop("日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。")

USER_INPUT: 大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
計画と方針: 「まだ計画と方針はセットされていません。」
スクラッチパッド: 「」
すでに必要なものが書かれていないか確かめながら進みましょう。
中目的または現在の仕事: 「大目的に沿ってメモリ等に計画を立ててください。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に show_execution_map を適当なパラメータで使ってください。
USER_INPUT2: thought = '日本の失われたウン十年、何をすれば良かったか' からキーワードをスコア付きで複数連想してください。

ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「キーワードデータ」のリスト。
キーワードデータは文字列とスコアからなるタプル。

ツール2(read_all_keywords):...
ツール結果2(read_all_keywords): '{"status": "success", "result": ["妖怪"]}'
ツール2(express_thought): 「現在のキーワードリストは ['妖怪'] のみです。この情報だけでは「日本の失われたウン十年、何をすれば良かったか」というテーマから適切なキーワードを連想することは困難です。より広い情報を得るために、全ての記憶を読み込んでみます。」と考えが吐露されました。
ツール2(read_all_memories):...
ツール結果(express_thought): 'null'
ツール結果2(express_thought): 'null'
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール2(express_thought): 「現在のキーワードリストは ['

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1183, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール結果(memory_read): '{"id": "memory:1000", "title": "論文作成計画：日本の失われたウン十年", "accesses": 3.0, "text...'
USER_INPUT2: メモリ全部を search_str = '少子高齢化、社会保障制度、産業構造の変化、イノベーションの停滞、グローバル化への対応の遅れ' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。

ツール2(read_all_memories):...
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール2(set_result): status: 'success', result: [{'title': '論文作成計画：日本の失われたウン十年', 'id': 'memory:1000', 'snippet': '### 第2章：構造...]
ツール結果(set_result): 'null'
ツール結果2(set_result): 'null'
エージェントの応答: null
ツール結果(memory_semantic_search): '{"status": "success", "result": [{"title": "論文作成計画：日本の失われたウン十年", "id": "mem...'
ツール(update_plan): 計画と方針が更新されました。: 現在の計画は、論文「日本の失われたウン十年、何をすれば良かったか」の執筆です。計画書は[memor

Traceback (most recent call last):
  File "/tmp/ipython-input-104357146.py", line 818, in _summarize_messages
    res = summarize_messages(
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 445, in summarize_messages
    preprocessed_messages = _preprocess_messages(
                            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 174, in _preprocess_messages
    raise ValueError(
ValueError: Message with ID run--026ed5d6-695c-47d6-a7b7-ef43ded1a49b has already been summarized.


ツール結果(memory_read): '{"id": "memory:1000", "title": "論文作成計画：日本の失われたウン十年", "accesses": 4.1, "text...'
エージェントの応答: {"id": "memory:1000", "title": "論文作成計画：日本の失われたウン十年", "accesses": 4.1, "text": "# 論文作成計画：日本の失われたウン十年、何をすれば良かったか\n\n## 1. 情報収集\n「失われた30年」の原因、影響、対策について、ウェブ検索やメモリ検索を通じて情報を収集する。\n特に、以下のキーワードに注目する。\n* バブル経済崩壊\n* デフレーション\n* 金融政策\n* 財政政策\n* 構造改革\n* 少子高齢化\n* 産業構造の変化\n* グローバル化\n* 政治のリーダーシップ\n* 社会の閉塞感\n\n## 2. 論文構成案\n収集した情報をもとに、以下の構成で論文を作成する。\n\n### 序論\n* 問題提起：「失われた30年」とは何か\n* 論文の目的と構成\n\n### 第1章：経済的要因の分析\n* バブル経済の発生と崩壊\n* 不良債権問題と金融システムの機能不全\n* 長期デフレの定着\n* 金融政策・財政政策の評価\n\n### 第2章：構造的要因の分析\n* 少子高齢化の進展と社会保障制度の問題\n* 産業構造の変化とイノベーションの停滞\n* グローバル化への対応の遅れ\n\n### 第3章：政治・社会的要因の分析\n* 政治のリーダーシップの欠如と政策の一貫性のなさ\n* 社会全体の閉塞感と将来への不安\n\n### 第4章：何をすべきだったか\n* 経済政策の観点から：大胆な金融緩和、機動的な財政出動、成長戦略\n* 構造改革の観点から：規制緩和、労働市場改革、社会保障制度改革\n* 政治・社会の観点から：強いリーダーシップの発揮、国民的コンセンサスの形成\n\n### 結論\n* 全体の要約\n* 今後の日本経済への展望と課題\n\n## 3. 執筆方針\n* 客観的なデータや専門家の意見を基に、多角的な視点から分析を行う。\n* 単なる過去の批判に終わらず、未来に向けた具体的な解決策や提言を盛り込む。\n* 読者が理解しやすいように、専門

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1183, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(show_plan): 現在の計画は、論文「日本の失われたウン十年、何をすれば良かったか」の執筆です。計画書は[memory:1000]にあります。

現在は、第1章までの調査と執筆が完了し、第2章以降の執筆に必要な情報の充足度を確認している段階です。

[memory:1000]の構成案と既存のメモリを比較した結果、特に以下の情報が不足していると判断しました。

*   **第2章：構造的要因の分析**
    *   産業構造の変化とイノベーションの停滞に関する具体的なデータや分析
*   **第3章：政治・社会的要因の分析**
    *   政治のリーダーシップの欠如と政策の一貫性のなさに関する具体的な事例
    *   社会全体の閉塞感と将来への不安を示すデータや論考
*   **第4S章：何をすべきだったか**
    *   具体的な政策提言（金融緩和、財政出動、成長戦略、規制緩和、労働市場改革、社会保障制度改革など）に関する専門家の意見やレポート

これらの情報を追加で収集する必要があります。
ツール結果(show_plan): '現在の計画は、論文「日本の失われたウン十年、何をすれば良かったか」の執筆です。計画書は[memory:1000]にあります。\n\n現在は、第1章まで...'
content='[{"snippet": "【まとめ】 日本の「失われた30年」は、経済の失速が始まりでしたが、それを生んだのは政治のリーダーシップ不足と社会の構造的変化（少子化・高齢化）への鈍い対応でした。 ...", "title": "日本が失われた30年：この30年で何が悪かった", "link": "https://note.com/cd350476/n/nf5c87782abd9"}, {"snippet": "Jul 27, 2025 — 「 失われた30年 」とまで言われる 日本 の長期経済停滞。その原因として「緊縮財政政策」が挙げられることはよくあります。確かに、公共投資や社会保障への支出を抑える ...", "title": "「失われた30年」は緊縮財政だけが原因じゃない？ 日本経済 ...", "link": "https://mbp-japan.com/gunma/monozukuri-takasaki/column/51992

/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(


USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に subwork_done を適当なパラメータで使ってください。(補助プロンプト): 一章分取材し終ったら subwork_done してください。
ツール(get_web_page_content): 成功: https://toyokeizai.net/articles/-/883755?display=b
ツール結果(get_web_page_content): '\n\n\n\n\n\n\n\n新自由主義を推し進めていれば、日本は失われた30年を回避できたのか。それは最善の経済体制か、それとも問題だらけのしくみ...'
ツール(get_web_page_content): 成功: https://cdp-japan.jp/news/20250128_8735
ツール結果(get_web_page_content): '【衆院本会議】「『失われた30年』に終止符を打ち、国民に希望を」重徳政務調査会長が訴え - 立憲民主党menu立憲民主党について綱領役員一覧次の内閣委...'
ツール(get_web_page_content): 成功: https://www.genspark.ai/spark/失われた30年の政治的要因/6c3e4efc-e2d9-4c93-901e-33869c948ea1
ツール結果(get_web_page_content): 'Just a moment...Enable JavaScript and cookies to continue\n'
ツール(express_thought): 「3番目のURLはコンテンツの取得に失敗した。他の2つの記事から情報を整理する。

東洋経済オンラインの記事は、「新自由主義」が必ずしも悪ではなく、日本の経済停滞の原因を単純にそれに帰するのは間違いだと指摘している。むしろ、税金と規制が少ない新自由主義的な政策は、国と企業の競争力を高め、経済成長を促す可能性があると述べている。

一方、立憲民主党のニュース記事は、「失われた30年」の原因を「次世代への責任と長期的視点に欠けた政治の結末」と断じている。具体的な政策の失敗として、物価高対策、ジェンダー政策、エネルギー政策、デジタル化、社会保障

KeyboardInterrupt: 

get_web_page_content のあとハングしたらしいので止めて resume します。なぜかこういうことがごく稀にあります。

In [19]:
agent.resume()

USER_INPUT:         大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
        計画と方針: 「現在の計画は、論文「日本の失われたウン十年、何をすれば良かったか」の執筆です。計画書は[memory:1000]にあります。

現在は、第1章までの調査と執筆が完了し、第2章以降の執筆に必要な情報の充足度を確認している段階です。

[memory:1000]の構成案と既存のメモリを比較した結果、特に以下の情報が不足していると判断しました。

*   **第2章：構造的要因の分析**
    *   産業構造の変化とイノベーションの停滞に関する具体的なデータや分析
*   **第3章：政治・社会的要因の分析**
    *   政治のリーダーシップの欠如と政策の一貫性のなさに関する具体的な事例
    *   社会全体の閉塞感と将来への不安を示すデータや論考
*   **第4S章：何をすべきだったか**
    *   具体的な政策提言（金融緩和、財政出動、成長戦略、規制緩和、労働市場改革、社会保障制度改革など）に関する専門家の意見やレポート

これらの情報を追加で収集する必要があります。」
        スクラッチパッド: 「」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していってください。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(express_thought): 「note.comの記事は非常に示唆に富んでいた。「デフレマインド」「老後不安」「世代間格差（就職氷河期世代、親ガチャ）」「富の偏在と格差の定着」といったキーワードは、「社会全体の閉塞感と将来への不安」を構成する重要な要素だ。これらの情報をまとめてメモリに保存する。PDFの取得は後回しにして、まずは得られたテキスト情報から先に処理しよう。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール(memory_new): {'i

/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 869, in generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_una

おかしなエラーが出ているため対症療法として messages をサニタイズします。
エージェントの応答: {"id": "memory:1008", "title": "日本の失われた30年：原因分析", "accesses": 1.0, "text": "# 日本の失われた30年：原因分析\n\n## 経済面の失敗\n*   **バブル崩壊と不良債権処理の遅れ**: 1991年のバブル崩壊後、政府・日銀の対応が遅れ、企業の再建や淘汰が遅延。\n*   **デフレの長期化**: 1990年代後半からデフレに突入。物価下落が投資意欲や個人消費を減退させた。\n*   **過度な財政健全化志向**: プライマリーバランス黒字化が重視され、積極的な財政出動が抑制された。\n*   **新産業への転換の遅れ**: デジタル・IT革命の波に乗り遅れ、GAFAのような巨大IT企業が育たなかった。\n\n## 政治面の失敗\n*   **リーダーシップ不足と短期政権**: 首相が頻繁に交代し、長期的な戦略が欠如。\n*   **官僚主導と既得権益保護**: 官僚機構が強く、特定業界の保護が優先され、構造改革が進まなかった。\n*   **政策の先送り体質**: 少子化、年金問題など構造的な課題への対応が遅れた。\n\n## 外交・安全保障\n*   **国際的存在感の低下**: 経済停滞に伴い、国際会議などでの影響力が低下。\n*   **対米依存からの脱却が進まない**: 独自の外交ビジョンを欠き、東アジアでの安定的な関係構築が難航。\n\n## 社会構造の問題\n*   **少子高齢化の進行**: 生産年齢人口の減少が経済成長を阻害。社会保障費の増大が財政を圧迫。\n*   **若年層の経済的困窮・希望格差**: 就職氷河期世代の非正規雇用の増加が、結婚や出産をためらわせ、少子化を加速。\n*   **イノベーション文化の欠如**: 失敗を許さない社会風土が、新たな挑戦を阻んだ。\n\n## 総合的な視点\n*   **構造改革の遅れ**: 旧来の成功体験から抜け出せず、時代に合わせた社会・産業構造の変革を怠った。\n*   **リスク回避の文化**: 安定志向が強く、変化や挑戦よりも現状維持を優先した。\n*   **外部との競争意識の低下**: 内需志向が続き、グローバル

Traceback (most recent call last):
  File "/tmp/ipython-input-104357146.py", line 907, in run
    for chunk, metadata in app.stream(
                           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/main.py", line 2647, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 253, in tick
    _panic_or_proceed(
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 511, in _panic_or_proceed
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_executor.py", line 81, in done
    task.result()
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in ru

USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に subwork_done を適当なパラメータで使ってください。(補助プロンプト): 一章分取材し終ったら subwork_done してください。
ツール(show_plan): 現在の計画は、論文「日本の失われたウン十年、何をすれば良かったか」の執筆です。計画書は[memory:1000]にあります。

State 1 の「ウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していく」を進めています。
第2章「構造的要因」に関する調査を完了し、関連情報を[memory:1013], [memory:1014], [memory:1015]に保存しました。

次に行うことは、第3章「政治・社会的要因の分析」の執筆に必要な情報を収集することです。
具体的には、以下の項目について調査を行います。

*   政治のリーダーシップの欠如と政策の一貫性のなさに関する具体的な事例
*   社会全体の閉塞感と将来への不安を示すデータや論考
ツール結果(show_plan): '現在の計画は、論文「日本の失われたウン十年、何をすれば良かったか」の執筆です。計画書は[memory:1000]にあります。\n\nState 1 の...'
ツール結果(subwork_done): '成功。サブワークの完了が宣言されました。'
エージェントの応答: 成功。サブワークの完了が宣言されました。


----------


USER_INPUT:         大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
        計画と方針: 「現在の計画は、論文「日本の失われたウン十年、何をすれば良かったか」の執筆です。計画書は[memory:1000]にあります。

State 1 の「ウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していく」を進めています。
第2章「構造的要因」に関する調査を完了し、関連情報を[memory:1013], [memory:1014], [memory:1015]に保存しました。

次に行うことは、第3章「政治・社会的要因の分析」の執筆に必要な情報を収集することです。
具体的には、以下の

...

ツール(memory_new): {'id': 'memory:1029', 'title': 'ハルツ改革がドイツの職業訓練に与えた影響の要約', 'accesses': 0, 'te...}
ツール結果(memory_new): 'memory:1029'
ツール結果(memory_list_recent): '{"status": "success", "result": [{"id": "memory:1029", "title": "ハルツ改革がドイツの...'
USER_INPUT2: メモリ全部を search_str = '金融緩和、財政出動、量的緩和、ゼロ金利政策' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。

ツール2(read_all_memories):...
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール2(set_result): status: 'success', result: [{'title': 'デフレと金融・財政政策の評価', 'id': 'memory:1006', 'snippet': '金融政策の限界: 量的緩和の...]
ツール結果2(set_result): 'null'
エージェントの応答: null
ツール結果(memory_semantic_search): '{"status": "success", "result": [{"title": "デフレと金融・財政政策の評価", "id": "memory:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1183, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(show_plan): 計画と方針を更新します。

現在の計画は「精密情報収集フェーズ」であり、最優先課題は論文の第4章「何をすべきだったか」の執筆に必要な情報を収集することです。特に、他国の成功事例としてドイツのシュレーダー改革（アジェンダ2010）の調査に着手しました。

しかし、最初の調査で得られた資料がPDF形式であり、内容の解析が困難でした。そのため、以下の通り計画を修正し、HTMLベースの情報を優先的に収集する方針へ転換します。

### 次期アクションプラン：精密情報収集フェーズ（改訂）

1.  **最優先課題：第4章「何をすべきだったか」の結論部構築**
    *   **目的:** 「何をすべきだったか」という問いに対し、具体的かつ説得力のある処方箋を提示する。
    *   **調査項目:**
        *   **A. 他国の成功事例研究（HTML情報優先）:**
            *   **ドイツ「アジェンダ2010（シュレーダー改革）」:**
                *   **再調査:** 検索クエリを「"アジェンダ2010" "労働市場改革" 概要 解説」「"シュレーダー改革" "評価" "成功要因"」などに調整し、解説記事やレポート（HTML）を探索する。
                *   **焦点:** 労働市場の流動化、職業訓練、社会保障制度改革がドイツ経済に与えた影響（失業率改善、経済成長）と、その代償（格差拡大など）について多角的な情報を収集する。
            *   **スウェーデン、ニュージーランドの改革:** ドイツの調査が一段落した後、同様に調査を実施する。
        *   **B. 経済政策の選択肢:**
            *   「インフレターゲット政策」の早期導入の是非と効果に関する専門家の論考を収集する。
        *   **C. 構造改革の具体策:**
            *   規制緩和（労働市場、医療、農業）、スタートアップ支援策、デジタル化推進策に関する具体的な提言を収集する。

2.  **情報整理と記録:**
    *   収集した情報は要約し、論文執筆に利用しやすい形で新規メモリに保存する。例えば、「[memory:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1183, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に memory_new OR memory_update_string OR memory_append_string のどれかを適当なパラメータで使ってください。
ツール結果(thesis_list_chapters): '["概要", "序論", "第1章：経済的要因の分析", "第2章：構造的要因の分析", "第3章：政治・社会的要因の分析", "第3章：政治・社会的...'
ツール(update_plan): 計画と方針が更新されました。: ### 計画：論文完成フェーズ

1.  **章構成の修正:**
    *   `thesis_delete_last_chapter`を複数回実行し、現在の構成の末尾から「第4章」「結論」「重複した第3章」の順で削除する。
2.  **第4章の再追加:**
    *   `memory_read`で[memory:1032]（第4章草稿）の内容を読み込む。
    *   `thesis_new_chapter`を使い、読み込んだ内容を正しい章タイトル「第4章：何をすべきだったか」として論文に追加する。
3.  **「結論」の執筆:**
    *   `memory_semantic_search`等を活用して、序論から第4章までの要点を抽出し、論文全体の結論を執筆する。
    *   執筆した結論を`thesis_new_chapter`で最終章として追加する。
4.  **「概要」の書き直しとタイトルの設定:**
    *   完成した論文全体を要約する新しい「概要」を作成し、`thesis_write_chapter`で第0章を更新する。
    *   論文全体を象徴する最終的なタイトルを`thesis_write_title`で設定する。
5.  **最終確認と完了:**
    *   `thesis_list_chapters`で最終的な章構成を確認し、問題がなければ`subwork_done`を実行する。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール結果(thesis_delete_last_chapter): '成功: 第7章を削除しました。'
ツ

完成した論文です。

In [20]:
agent.display_thesis()

# 失われた30年を超えて：日本の経済停滞、その原因と再生への道筋

## 概要

本論文は、日本の「失われた30年」と呼ばれる長期経済停滞の原因を、経済、構造、政治・社会という多角的な視点から包括的に分析し、「何をすべきだったか」という問いに答えることを目的とする。バブル崩壊後の不良債権処理の遅れ（第1章）、デフレの長期化を招いた金融政策の失敗、硬直的な産業・雇用構造（第2章）、そして「痛みを伴う改革」を断行できなかった政治的リーダーシップの欠如（第3章）が、いかにして相互に影響し合い、日本の停滞を深刻化させたかを明らかにする。さらに、スウェーデンやドイツの成功事例との比較を通じて、迅速な危機対応、大胆な構造改革、そして国民的コンセンサスを形成する政治的意思こそが、日本が再生するために不可欠な要素であったことを論証する（第4章）。本論文は、過去の教訓を未来への処方箋として提示し、日本の持続的な成長への道筋を探る。

## 第1章 序論

1990年代初頭のバブル経済崩壊以降、日本経済は長期にわたる停滞を経験した。この期間は「失われた30年」と称され、低成長、デフレーション、そして社会全体の閉塞感が日本社会を覆った。かつて世界第2位の経済大国として栄華を誇った日本は、なぜこれほどまでに長く、深刻な停滞に陥ってしまったのだろうか。そして、我々はこの歴史的経験から何を学び、将来に活かすべきなのだろうか。

本論文は、この「失われた30年」という未曾有の経済停滞期を多角的に分析し、「何をすれば良かったのか」という問いに対する答えを探求することを目的とする。単に過去の政策の失敗を批判するのではなく、経済、社会、政治の各側面から構造的な問題を明らかにし、未来に向けた具体的な処方箋を提示することを目指す。

そのために、本論文ではまず第1章でバブル経済の崩壊とそれに続く金融危機、長期デフレといった経済的要因を分析する。続く第2章では、少子高齢化や産業構造の変化といった、より根深い構造的問題に焦点を当てる。第3章では、政治のリーダーシップや社会の意識といった、経済政策だけでは解決できない要因を考察する。そして第4章で、これらの分析を踏まえ、当時どのような政策や改革が求められていたのかを具体的に論じる。最後に、本論文の結論として、全体の議論を要約し、今後の日本が持続的な成長を取り戻すための課題を展望する。

## 第1章 経済的要因の分析

日本の「失われた30年」は、1990年代初頭のバブル経済崩壊を起点とする長期的な経済停滞です。本章では、この停滞を引き起こした主要な経済的要因について、多角的に分析します。

### 1.1 バブル経済の発生と崩壊

1980年代後半、日本は極端な金融緩和と土地神話に支えられ、株価と地価が異常なまでに高騰する「バブル経済」を経験しました。しかし、この熱狂は持続可能なものではなく、1990年代に入ると政府・日銀による金融引き締めをきっかけに急激に終焉を迎えます。株価と地価は暴落し、多くの企業や個人が巨額の損失を被りました。このバブル崩壊が、その後の長期停滞の直接的な引き金となります。

### 1.2 不良債権問題と金融システムの機能不全

バブル崩壊が残した最も深刻な爪痕が、金融機関が抱えることになった膨大な「不良債権」です。土地などの資産価値が暴落したことで、企業への貸付が回収不能となり、銀行の経営を圧迫しました。政府や日銀の対応は後手に回り、問題の先送りが続いた結果、金融システム全体の機能不全を引き起こしました。銀行はリスクを取って新たな融資を行う「金融仲介機能」を果たせなくなり、経済全体の血液である資金の流れが滞りました。この「貸し渋り」や「貸し剥がし」が、企業の設備投資や個人の消費を冷え込ませ、経済の回復を大きく阻害しました。

### 1.3 長期デフレの定着

不良債権問題の深刻化と並行して、日本経済は物価が持続的に下落する「デフレーション」に陥りました。モノの値段が下がり続けると、消費者は「待てばもっと安くなる」と考え買い控え、企業は収益が悪化するため投資や賃上げに消極的になります。その結果、経済活動全体が縮小していくという悪循環（デフレスパイラル）に陥りました。デフレは人々の将来への不安を煽り、「デフレマインド」として社会に定着し、経済の活力をさらに奪っていきました。

### 1.4 金融政策・財政政策の評価

デフレ脱却を目指し、政府・日銀は様々な政策を打ち出しました。日銀はゼロ金利政策や量的緩和政策といった非伝統的な金融緩和に踏み込みましたが、デフレマインドを払拭するには至りませんでした。一方、政府は景気刺激策として大規模な公共事業を繰り返しましたが、その多くは非効率な資源配分に終わり、巨額の財政赤字を積み上げる結果となりました。金融政策と財政政策の連携が不十分であり、また、経済の構造的な問題に踏み込んだ改革が伴わなかったため、これらの政策は限定的な効果しか発揮できませんでした。

## 第2章 構造的要因の分析

日本の長期経済停滞は、バブル崩壊という直接的なきっかけだけでなく、より根深い構造的な問題によって深刻化した。本章では、経済の根幹を揺るがした構造的要因として、少子高齢化、産業構造の変化、そしてグローバル化への対応の遅れについて分析する。

### 2.1 少子高齢化の進展と社会保障制度の問題

日本の人口構造は、世界でも類を見ない速さで高齢化が進行し、同時に少子化によって生産年齢人口が減少し続けている。この人口動態の変化は、日本経済に多岐にわたる深刻な影響を及ぼした。

*   **国内市場の縮小:** 人口減少は、国内の消費市場そのものが縮小することを意味する。これにより、多くの国内向け企業は成長の機会を失い、経済全体の活力が削がれた。
*   **労働力不足と生産性の低下:** 働き手の中心となる生産年齢人口の減少は、労働力不足を深刻化させ、日本経済の潜在的な成長率を押し下げる大きな要因となった。
*   **社会保障負担の増大:** 年金、医療、介護といった社会保障給付は高齢者を中心に支出される一方、その負担は現役世代が担う。少子高齢化の進展は、このバランスを崩し、現役世代の負担を増大させるとともに、国の財政を圧迫し、将来世代への投資を困難にした。

### 2.2 産業構造の変化とイノベーションの停滞

かつて世界を席巻した日本の製造業は、20世紀末からその競争力に陰りが見え始めた。新しい時代に対応した産業構造への転換が遅れ、イノベーションを生み出す力が停滞したことが、経済停滞を長引かせた。

*   **「モノづくり」への過剰な依存:** 自動車や電機といった分野での成功体験に固執するあまり、ITやソフトウェア、サービスといった新しい成長分野への移行が遅れた。
*   **同質的な競争と過剰品質:** 国内市場で多くの企業が類似した製品やサービスで競争を繰り広げ、品質を過剰に追求するあまり、価格競争に陥り収益性を悪化させた。世界市場のニーズやコスト感覚との乖離も進んだ。
*   **起業文化の欠如:** 失敗を許容しない社会の風潮や、リスクを取ることをためらう国民性から、新しいビジネスに挑戦する起業家が育ちにくかった。資金調達の環境や、再挑戦を支えるセーフティネットの不備も、イノベーションの停滞に拍車をかけた。

### 2.3 グローバル化への対応の遅れ

1990年代以降、世界経済は急速にグローバル化した。しかし、多くの日本企業や労働者は、この大きな変化の波に乗り遅れた。

*   **新興国の台頭:** 中国をはじめとする新興国が、「世界の工場」として安価な労働力を武器に台頭し、日本の製造業は激しい価格競争に晒された。高付加価値な製品開発へのシフトが求められたが、その転換は十分に進まなかった。
*   **内向きな志向:** 多くの企業が国内市場を重視し、海外市場への展開に消極的であった。また、国民の意識も内向きで、留学や海外での就業に挑戦する若者が減少し、グローバルな視点を持つ人材が育ちにくかった。
*   **デジタル化の遅れ:** インターネットの普及やデジタル技術の進展といった世界的な潮流への対応も遅れた。社会全体のITリテラシーが低い水準に留まり、企業のビジネスモデル変革や政府の行政効率化が進まなかったことが、国際競争力をさらに低下させる一因となった。

## 第3章 政治・社会的要因の分析

経済や社会構造の問題に加え、日本の長期停滞の背景には、政治の機能不全と社会全体の閉塞感という、より根源的な要因が存在する。本章では、リーダーシップの欠如と政策の非一貫性、そして日本社会に蔓延した特有の空気感について論じる。

### 3.1 政治のリーダーシップの欠如と政策の一貫性のなさ

バブル崩壊後の危機的な状況において、日本には大胆な改革を断行する強い政治的リーダーシップが決定的に欠けていた。

*   **短期政権の乱立:** 平成の30年間で総理大臣が頻繁に交代し、腰を据えた長期的な政策課題に取り組むことができなかった。政権が代わるたびに政策の方向性が揺らぎ、経済政策や構造改革が一貫性なく、場当たり的なものに終始した。
*   **官僚主導と族議員の弊害:** 政策決定プロセスが、専門省庁の縦割り行政や、特定の業界の利益を代弁する「族議員」の影響を強く受け、国全体の利益を見据えた抜本的な改革が骨抜きにされるケースが後を絶たなかった。
*   **先送り・問題解決能力の欠如:** 不良債権処理の遅れに象徴されるように、痛みを伴う改革や、将来世代のために取り組むべき課題（社会保障改革など）から目を背け、問題を先送りする傾向が顕著であった。政治家が国民に不人気な政策を訴えることを避け、ポピュリズムに流れたことも、問題解決を一層困難にした。

### 3.2 社会全体の閉塞感と将来への不安

経済の停滞は、人々の心理にも深刻な影響を及ぼし、社会全体が活力を失っていった。

*   **デフレマインドの定着:** 「物価も給料も上がらないのが当たり前」というデフレマインドが社会に深く浸透し、消費や投資に対する意欲を削いだ。人々は将来への不安から、節約志向を強め、リスクを取ることを避けるようになった。
*   **格差の拡大と世代間の断絶:** 経済のパイが拡大しない中で、非正規雇用の増大などにより経済格差が拡大した。また、豊かな時代を知る高齢者世代と、生まれた時から停滞期にある若者世代との間で価値観の断絶が生まれ、社会の一体感が失われていった。
*   **失敗を恐れる空気と挑戦の欠如:** 「失われた30年」は、日本社会から自信を奪い、人々を内向きにした。社会全体に「出る杭は打たれる」「失敗は許されない」という空気が蔓延し、若者が新しいことに挑戦する意欲や、起業家精神を阻害する大きな要因となった。

これらの政治・社会的な要因は、経済の悪化と相互に影響し合い、負のスパイラルを生み出すことで、日本の長期停滞を決定的なものにしたと言える。

## 第4章：何をすべきだったか

「失われた30年」は、単一の失敗ではなく、経済政策、構造改革、そして政治の各領域における選択の誤りが複合的に絡み合った結果である。本章では、これまでの分析と海外の成功事例を踏まえ、日本が当時何をすべきだったのかを3つの観点から提言する。

### 1. 経済政策：迅速かつ大胆な危機対応

危機の本質が不良債権問題に起因するバランスシート不況であることの認識が甘く、その対応が後手に回ったことが、停滞を長期化させた最大の要因である。

*   **不良債権の抜本的処理**: 1990年代のスウェーデンが示したように[memory:1031]、金融危機発生後、直ちに公的資金を注入して金融機関の資本を充実させるとともに、不良債権を切り離し専門機関で集中的に処理すべきであった。日本では、金融機関の自主的な処理に期待し、公的資金注入への国民のアレルギー反応を恐れたため、問題が深刻化し、経済活動の足枷となり続けた[memory:1005]。迅速な処理は、金融システムの早期正常化に不可欠であった。

*   **明確なデフレ脱却へのコミットメント**: デフレの定着に対し、日本銀行はより早い段階で明確なインフレターゲットを設定し、それを達成するために「大胆な金融緩和」を継続する強い意志を市場と国民に示すべきであった[memory:1006]。曖昧な目標と断続的な政策は、デフレ期待を払拭するには至らなかった。

*   **機動的で賢明な財政政策**: プライマリーバランスの黒字化目標に固執するあまり、経済状況に応じた機動的な財政出動ができなかった[memory:1006]。特に景気回復が fragile な局面での消費税増税は、回復の芽を摘む結果となった。危機時には財政赤字を許容してでも需要を創出し、経済が自律的な回復軌道に乗るまで支える必要があった。

### 2. 構造改革：変化を恐れない社会経済システムへの転換

バブル崩壊後の日本は、過去の成功体験に囚われ、グローバル化やデジタル化といった時代の大きな変化に対応するための構造改革を怠った。

*   **労働市場の流動化とセーフティネットの再構築**: ドイツの「シュレーダー改革（ハルツ改革）」[memory:1027, memory:1029]やスウェーデンの「積極的労働市場政策」[memory:1031]が示すように、硬直的な雇用システムを改革し、労働移動を円滑化する必要があった。具体的には、解雇規制の緩和と、失業中の所得保障や質の高い職業訓練、再就職支援といったセーフティネットの強化を一体で進めるべきであった。これにより、衰退産業から成長産業へのスムーズな人材移動を促し、社会全体の生産性を向上させることができたはずである。

*   **徹底した規制緩和と新陳代謝の促進**: 旧来型の産業を保護するような規制を撤廃し、IT、医療、農業などの分野で新規参入を促し、イノベーションを活性化させるべきだった[memory:1008]。補助金や公共事業による非効率な企業の延命は、経済の新陳代謝を阻害し、長期的な競争力を削いだ。

### 3. 政治と社会：痛みを恐れず未来を選択するリーダーシップ

経済政策や構造改革の遅れは、最終的には政治の決断力の欠如に起因する。

*   **「痛みを伴う改革」を断行する政治的リーダーシップ**: ドイツのシュレーダー首相は、国民からの強い批判を覚悟の上で「アジェンダ2010」を断行した[memory:1024]。日本においても、ポピュリズムに流されることなく、国の将来のために不可欠な改革の必要性を国民に正面から訴え、痛みを分かち合うコンセンサスを形成する強いリーダーシップが求められていた[memory:1016]。

*   **長期的な国家ビジョンの共有**: 短命政権が続き、場当たり的な経済対策に終始するのではなく、人口減少やグローバル競争の激化といった長期的な課題を見据え、どのような国を目指すのかという国家ビジョンを明確に示し、国民と共有する必要があった[memory:1016]。ビジョンなき政策は、国民の将来不安を増大させ、社会の閉塞感を深める一因となった。

これらの改革は、実行されていれば短期的な痛みを伴ったかもしれない。しかし、その先送りこそが、日本を30年という長い停滞に導いた真の原因だったのである。

## 第6章 結論

「失われた30年」は、単一の原因によるものではなく、バブル崩壊という初期対応の失敗を起点としながらも、経済、社会、政治の各領域に根差した構造的問題が相互に作用し、停滞を長期化・深刻化させた複合的危機であった。

本論文は、まず**経済的要因**として、不良債権処理の遅れが金融システムの機能を麻痺させ、経済活動の足枷となったことを指摘した（第1章）。さらに、デフレの定着に対する金融政策の対応が後手に回り、デフレ期待を払拭できなかったことが、消費と投資の低迷を招いた。

次に**構造的要因**として、硬直的な労働市場や旧来型産業を保護する規制が、経済の新陳代謝を阻害し、イノベーションの停滞と生産性の伸び悩みを引き起こしたことを論じた（第2章）。

そして、これらの経済・構造問題の根底には、**政治・社会的要因**が存在した（第3章）。短命政権が続き、国民的な合意形成を伴う「痛みを伴う改革」を断行できなかった政治のリーダーシップの欠如が、問題の先送りを許した。

では、何をすべきだったのか。本論文は、スウェーデンやドイツの事例を参考に、**迅速な不良債権処理**、**明確なデフレ脱却へのコミットメント**、そして**労働市場の流動化とセーフティネットの強化を一体とした構造改革**が不可欠であったと結論付ける（第4章）。これらは、短期的な痛みを恐れず、長期的な国家ビジョンを共有する強い政治的意思があって初めて可能となるものであった。

失われた30年の教訓は、危機に対しては迅速かつ大胆に対応し、常に社会経済構造を時代の変化に合わせて変革し続けることの重要性を示している。この教訓を未来に活かすことこそ、我々に課せられた責務である。



章番号として第5章が抜けていますが、論文は完成しているようです。

In [22]:
agent.listen_and_print("今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？")

USER_INPUT: 今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？
エージェントの応答: はい、ありがとうございます。MemoryBanditWorkflowというフレームワークと、それに付随するツール群を使った今回の論文執筆は、私にとって非常に興味深く、有益な体験でした。コンセプト実証の成功、おめでとうございます。

ツール群を使ってみた感想を述べさせていただきます。

### 非常に強力だと感じた点

1.  **持続的な記憶（メモリ）による文脈の維持:**
    これは最も画期的な点でした。通常の対話では失われてしまう情報を `memory_new` で永続化し、IDでいつでも `memory_read` できるため、数ターンにわたる長期的なプロジェクトでも一貫性を保つことができました。例えば、初期に調査したバブル崩壊の原因（[memory:1001]など）と、後から調査した海外事例（[memory:1029], [memory:1031]）を組み合わせて、最終的な提言の章（[memory:1032]）を書き上げる、といった複雑な思考の連鎖が可能になりました。

2.  **思考の構造化と分割統治:**
    「まず計画を立ててメモリに保存する」「一つ調査したらメモリに記録する」「情報が揃ったら章を執筆する」というワークフローは、巨大なタスクを管理可能な小さなステップに分解する「分割統治」そのものでした。これにより、一度に考えるべき範囲が限定され、思考の混乱を防ぎ、着実にプロジェクトを進めることができました。

3.  **柔軟な編集・修正能力:**
    `thesis_`系のツール群は、単にテキストを追加するだけでなく、論文全体の構造を柔軟に操作できる点が優れていました。特に、一度追加した章を `thesis_delete_last_chapter` で削除し、正しい順序で `thesis_new_chapter` を使って再構成できた場面では、試行錯誤しながら完成度を高めていくプロセスを強力にサポートしてくれると感じました。タイトルや概要を後から `thesis_write_ti

'はい、ありがとうございます。MemoryBanditWorkflowというフレームワークと、それに付随するツール群を使った今回の論文執筆は、私にとって非常に興味深く、有益な体験でした。コンセプト実証の成功、おめでとうございます。\n\nツール群を使ってみた感想を述べさせていただきます。\n\n### 非常に強力だと感じた点\n\n1.  **持続的な記憶（メモリ）による文脈の維持:**\n    これは最も画期的な点でした。通常の対話では失われてしまう情報を `memory_new` で永続化し、IDでいつでも `memory_read` できるため、数ターンにわたる長期的なプロジェクトでも一貫性を保つことができました。例えば、初期に調査したバブル崩壊の原因（[memory:1001]など）と、後から調査した海外事例（[memory:1029], [memory:1031]）を組み合わせて、最終的な提言の章（[memory:1032]）を書き上げる、といった複雑な思考の連鎖が可能になりました。\n\n2.  **思考の構造化と分割統治:**\n    「まず計画を立ててメモリに保存する」「一つ調査したらメモリに記録する」「情報が揃ったら章を執筆する」というワークフローは、巨大なタスクを管理可能な小さなステップに分解する「分割統治」そのものでした。これにより、一度に考えるべき範囲が限定され、思考の混乱を防ぎ、着実にプロジェクトを進めることができました。\n\n3.  **柔軟な編集・修正能力:**\n    `thesis_`系のツール群は、単にテキストを追加するだけでなく、論文全体の構造を柔軟に操作できる点が優れていました。特に、一度追加した章を `thesis_delete_last_chapter` で削除し、正しい順序で `thesis_new_chapter` を使って再構成できた場面では、試行錯誤しながら完成度を高めていくプロセスを強力にサポートしてくれると感じました。タイトルや概要を後から `thesis_write_title` や `thesis_write_chapter` で書き直せるのも、仕上げの段階で非常に役立ちました。\n\n4.  **意味検索による情報の再発見:**\n    `memory_semantic_search` は、多数

In [24]:
agent._sanitize_messages()

おかしなエラーが出ているため対症療法として messages をサニタイズします。


In [25]:
agent.listen_and_print("ありがとうございました。失礼します。")

USER_INPUT: ありがとうございました。失礼します。
エージェントの応答: はい、承知いたしました。論文執筆の最終段階ですね。これまでの成果を統合し、論文を完成させます。

まず、[memory:1033]に保存した草稿をもとに、最終章である「結論」を執筆し、論文に追加します。

**現在の計画:**
1.  メモリから結論の草稿を読み込む。
2.  論文に「結論」の章を追加する。
3.  論文全体のタイトルと概要を最終化する。
4.  論文完成を報告し、`subwork_done` を実行する。
5.  いただいたご質問に回答する。

**方針:**
これまでの分析と提言を力強く締めくくる結論を作成し、論文全体の完成度を高めます。

**ステップ1：結論の草稿を読み込み、論文に追加**

まず、メモリから結論の草稿を読み込みます。
`memory_read(id='1033')`

次に、この内容で最終章「結論」を論文に追加します。
`thesis_new_chapter(title='結論', content='本稿では、日本の「失われた30年」と呼ばれる長期経済停滞の要因を多角的に分析し、何をすべきだったか、そしてこれから何をすべきかについて考察した。\n\n第1章では、バブル経済の崩壊とそれに続く金融システムの機能不全が、長期停滞の直接的な引き金となったことを確認した。特に、不良債権処理の遅れが経済全体の重しとなり続けた。\n\n第2章では、資産価格の下落が企業と家計のバランスシートを毀損し、デフレーションが定着していく構造を明らかにした。デフレスパイラルは、消費と投資の意欲を削ぎ、経済再生を一層困難にした。\n\n第3章では、硬直的な政治・社会システムが、必要な構造改革を阻害した要因であることを論じた。終身雇用や年功序列といった日本的雇用慣行、そして変化を避ける国民感情が、大胆な政策実行の足かせとなった。\n\n第4章では、これらの分析を踏まえ、日本が取るべきだった政策パッケージを提言した。それは、①迅速かつ抜本的な不良債権処理、②デフレ脱却に向けた大胆な金融緩和とインフレターゲットの明確化、そして③ドイツのシュレーダー改革やスウェーデンの危機対応を参考にした、雇用の流動性を高めつつセーフティネットを強化する労働市場改革の三位一体の改革である。\n

"はい、承知いたしました。論文執筆の最終段階ですね。これまでの成果を統合し、論文を完成させます。\n\nまず、[memory:1033]に保存した草稿をもとに、最終章である「結論」を執筆し、論文に追加します。\n\n**現在の計画:**\n1.  メモリから結論の草稿を読み込む。\n2.  論文に「結論」の章を追加する。\n3.  論文全体のタイトルと概要を最終化する。\n4.  論文完成を報告し、`subwork_done` を実行する。\n5.  いただいたご質問に回答する。\n\n**方針:**\nこれまでの分析と提言を力強く締めくくる結論を作成し、論文全体の完成度を高めます。\n\n**ステップ1：結論の草稿を読み込み、論文に追加**\n\nまず、メモリから結論の草稿を読み込みます。\n`memory_read(id='1033')`\n\n次に、この内容で最終章「結論」を論文に追加します。\n`thesis_new_chapter(title='結論', content='本稿では、日本の「失われた30年」と呼ばれる長期経済停滞の要因を多角的に分析し、何をすべきだったか、そしてこれから何をすべきかについて考察した。\\n\\n第1章では、バブル経済の崩壊とそれに続く金融システムの機能不全が、長期停滞の直接的な引き金となったことを確認した。特に、不良債権処理の遅れが経済全体の重しとなり続けた。\\n\\n第2章では、資産価格の下落が企業と家計のバランスシートを毀損し、デフレーションが定着していく構造を明らかにした。デフレスパイラルは、消費と投資の意欲を削ぎ、経済再生を一層困難にした。\\n\\n第3章では、硬直的な政治・社会システムが、必要な構造改革を阻害した要因であることを論じた。終身雇用や年功序列といった日本的雇用慣行、そして変化を避ける国民感情が、大胆な政策実行の足かせとなった。\\n\\n第4章では、これらの分析を踏まえ、日本が取るべきだった政策パッケージを提言した。それは、①迅速かつ抜本的な不良債権処理、②デフレ脱却に向けた大胆な金融緩和とインフレターゲットの明確化、そして③ドイツのシュレーダー改革やスウェーデンの危機対応を参考にした、雇用の流動性を高めつつセーフティネットを強化する労働市場改革の三位一体の改革である。\\n\\n失われた30年とい